In [ ]:
#Expr class to store each expression
class Expr(object):
    def __init__(self,i,a,sigma,tau,j):
        self.i = i
        self.a = a
        self.sigma = sigma
        self.tau = tau
        self.j = j
    def __str__(self):
        return 's[' + str(self.i) + ']: ' + self.a + ' → ' + self.sigma + '•' + self.tau + ', ' + str(self.j)
        
#Data class to store each data of transitions
class Data(object):
    def __init__(self,method,hl,src,dest):
        self.method = method
        self.hl = hl
        self.src = src
        self.dest = dest
    def __str__(self):
        return self.method + '\n' + self.src.__str__() + '\n' + self.dest.__str__()

#define an empty list to store the data used for animation
graph_data = []

#Modified from lecture notes 11: gernerlized parsing
def parse(g: "grammar", x: "input"):
    global s
    n = len(x); x = '^' + x + '$'; S, π = g[0][0], g[0][2:]
    s = [{(S, '', π, 0)}] + [set() for _ in range(n)]#; print('   s[ 0 ]:', S, '→ •', π, ', 0')
    for i in range(n + 1):
        v = set() # visited items
        while v != s[i]:
            e = (s[i] - v).pop(); v.add(e) # pick an arbirary un-visited item
            A, σ, τ, j = e
            if len(τ) > 0 and τ[0] == x[i + 1]: # match, a == τ[0]
                f = (A, σ + τ[0], τ[1:], j)
                s[i + 1].add(f)
                #store animation data
                src = Expr(i, A, σ, τ, j)
                dest = Expr(i, f[0], f[1], f[2], f[3])
                graph_data.append(Data('Match', τ[0], src, dest))
            elif len(τ) > 0: # predict, B == ω[0]
                for f in ((r[0], '', r[2:], i) for r in g if r[0] == τ[0]):
                    s[i].add(f)
                    #store animation data
                    src = Expr(i, A, σ, τ, j)
                    dest = Expr(i, f[0], f[1], f[2], f[3])
                    graph_data.append(Data('Predict', f[0], src, dest))
            else: # complete, len(τ) == 0
                for f in ((B, μ + ν[0], ν[1:], k) for (B, μ, ν, k) in s[j] if len(ν) > 0 and ν[0] == A):
                    s[i].add(f)
                    #store animation data
                    src = Expr(i, A, σ, τ, j)
                    dest = Expr(i, f[0], f[1], f[2], f[3])
                    graph_data.append(Data('Complete', A, src, dest))
    return (S, π, '', 0) in s[n]

In [ ]:
G1 = ("S→E", "E→a", "E→E+E")
parse(G1, "a+a+a")

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import time

#set this to TRUE to auto-animate the process
auto_generate = False

#define a 'next' button
nxt = widgets.Button(description='next')
display(nxt)

class Animate(object):
    def __init__(self,data):
        self.graph_data = data
        self.i = 0
        
    #mark text at given index to red
    def mark_red(self,text,index):
        return text[:index]+"\x1b[31m"+text[index]+"\x1b[0m"+text[index:][1:]
        
    #function used for button onclick event
    def func(self,btn):
        if self.i < len(graph_data):
            method = self.graph_data[self.i].method
            src = self.graph_data[self.i].src
            dest = self.graph_data[self.i].dest
            print("\x1b[91m"+method+"\x1b[0m")
            if method == 'Predict':
                inx = src.tau.find(self.graph_data[self.i].hl)
                src.tau = self.mark_red(src.tau,inx)
                print(src)
                dest.a = self.mark_red(dest.a,0)
                print(dest)
            elif method == 'Match':
                src.tau = self.mark_red(src.tau,0)
                print(src)
                dest.sigma = self.mark_red(dest.sigma,len(dest.sigma)-1)
                print(dest)
            elif method == 'Complete':
                src.a = self.mark_red(src.a,0)
                print(src) 
                dest.sigma = self.mark_red(dest.sigma,len(dest.sigma)-1)
                print(dest)
            print()
            self.i += 1
        else:
            print("====FINISH====")

#initialize the Animate class
anime = Animate(graph_data)
if auto_generate:
    for i in range(len(graph_data)):
        time.sleep(1)
        anime.func(nxt)
else:
    nxt.on_click(anime.func)